In [1]:
# Detect how similar a resume is compared to a job description

In [2]:
pip install docx2txt

  Created wheel for docx2txt: filename=docx2txt-0.8-cp37-none-any.whl size=3981 sha256=92939127e23c663cf003c5e3853634eca11b2bdcb8b87a175a1b4bc0af2f2c59
  Stored in directory: /root/.cache/pip/wheels/b2/1f/26/a051209bbb77fc6bcfae2bb7e01fa0ff941b82292ab084d596
Successfully built docx2txt


In [3]:
import docx2txt
import nltk

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
# Setup
!pip install -q wordcloud
import wordcloud

import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize

import re
import subprocess  # noqa: S404
import os
import spacy
nlp = spacy.load('en_core_web_sm')
import pprint
from spacy.matcher import Matcher
import multiprocessing as mp
import glob
import docx2txt
# Load files
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping chunkers/maxent_ne_chunker.zip.
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Unzipping corpora/words.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already 

In [4]:
sw = ["pron","-PRON-","and","the","with","have","a","be","to","in","will","if","by","into","as","&","of","on","at","is",
              "for","then", "once", "here", "there","who" ,"when", "where", "why", "how","an","i",
              "about", "against", "between", "through", "during", "before", "after", "me", "my", "myself", "we", "our",
              "ours", "ourselves", "you", "you're", "you've","from","can","those","but","that","may","over","often","this","them","do","much"]

In [5]:
! pip install -q kaggle
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"asadbaig1","key":"575395854de15734d29e433921443ffe"}'}

In [6]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets list
! kaggle datasets download -d palaksood97/resume-dataset
! mkdir resume-dataset
! unzip resume-dataset.zip -d resume-datasetdf

ref                                                         title                                              size  lastUpdated          downloadCount  
----------------------------------------------------------  ------------------------------------------------  -----  -------------------  -------------  
gpreda/reddit-vaccine-myths                                 Reddit Vaccine Myths                              229KB  2021-06-01 11:18:46           6702  
crowww/a-large-scale-fish-dataset                           A Large Scale Fish Dataset                          3GB  2021-04-28 17:03:01           3980  
imsparsh/musicnet-dataset                                   MusicNet Dataset                                   22GB  2021-02-18 14:12:19           1345  
dhruvildave/wikibooks-dataset                               Wikibooks Dataset                                   1GB  2021-02-18 10:08:27           2094  
promptcloud/careerbuilder-job-listing-2020                  Careerbuilder Jo

In [7]:
def extract_text_from_docx(docx_path):
    txt = docx2txt.process(docx_path)
    if txt:
        return txt.replace('\t', ' ')
    return None

In [8]:
def get_doc_list(files):
    doc_list = []
    for file in files:
        print(file)
        st = extract_text_from_docx(file)
        doc_list.append(st)
    print ('Found %s documents under the dir %s .....'%(len(files),"/content/resume-datasetdf"))
    return doc_list

    # doc_list 
file_path = glob.glob("/content/resume-datasetdf/Resumes/*.docx")
documents = get_doc_list(file_path)

/content/resume-datasetdf/Resumes/Yohan BSA.docx
/content/resume-datasetdf/Resumes/Drakshajavauidev.docx
/content/resume-datasetdf/Resumes/Mounika health care resume.docx
/content/resume-datasetdf/Resumes/Sravani Singirikonda.docx
/content/resume-datasetdf/Resumes/Kumar Raj.docx
/content/resume-datasetdf/Resumes/Jimi_Desai_PM.docx
/content/resume-datasetdf/Resumes/Nikki_Nimmagadd.docx
/content/resume-datasetdf/Resumes/Saiteja G.docx
/content/resume-datasetdf/Resumes/Jaya prakash.docx
/content/resume-datasetdf/Resumes/Raju Goduguchinta_Technical Program Manager.docx
/content/resume-datasetdf/Resumes/KIRAN KUMAR.docx
/content/resume-datasetdf/Resumes/Siddhartha Gandroju.docx
/content/resume-datasetdf/Resumes/Sravani Battu.docx
/content/resume-datasetdf/Resumes/Amar Sr BSA.docx
/content/resume-datasetdf/Resumes/Navneeth Resume.docx
/content/resume-datasetdf/Resumes/Vinay.D_Resume.docx
/content/resume-datasetdf/Resumes/Syed_Zia_Ashraf.docx
/content/resume-datasetdf/Resumes/Madhuri Pawar.do

In [9]:
docs_list = []
for d in documents:
    d_nlp = nlp(d.lower())
    t_list = []
    for token in d_nlp:
        tok_lem = str(token.lemma_)
        if (tok_lem not in sw):
            t_list.append(tok_lem)
    str_ = ' '.join(t_list) 
    docs_list.append(str_)

In [11]:
# Store and print the resume
import docx2txt


def extract_text_from_docx(docx_path):
    txt = docx2txt.process(docx_path)
    if txt:
        return txt.replace('\t', ' ')
    return None


if __name__ == '__main__':
  job_description = extract_text_from_docx('/content/Job description.docx')
  print(job_description)

Job description



Overall years of experience in Software design and development activities including requirement analysis, design, implementations and testing of various client server based applications using Java and J2EE technologies. 

Extensive experience in Core Java, Servlets, JSP, JavaScript, JDBC, JSTL, Struts, JSF, EJB, AJAX and Custom Tag Libraries. 

Extensive experience in working with different MVC Frameworks like Spring, Struts and ORM (Object Relational mapping tools) like Hibernate. 

Experienced working with databases like DB2, MY SQL, Oracle, MS SQL Server and Application Servers like BEA WebLogic, IBM WebSphere, Apache Tomcat.

Expertise in developing critical & responsive GUI / front-end components using HTML, XHTML, DHTML, JavaScript, CSS2/CSS3, JSON, JSP, Servlets, Ajax, JQuery, EXT.JS, JSTL and JavaScript using frameworks like Bootstrap, AngularJS, Ext JS. 

Experience in application of MVC and MVVM client-side JavaScript frameworks such as Angular 2.0, Angular

In [12]:
# Store and print the job description
# job_description = docx2txt.process("pythonSampleJobDescription.docx")
# print(job_description)

In [13]:
# Create list of the two documents
text = [docs_list[0], job_description]

In [14]:
def find_between(s, start, end):
  return (s.split(start))[1].split(end)[0]

In [18]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
for i, t in enumerate(docs_list):
 text = [t, job_description]
 cv = CountVectorizer()
 count_matrix = cv.fit_transform(text)
#  print(cosine_similarity(count_matrix))
 # Get match percentage
 matchPercentage = cosine_similarity(count_matrix)[0][1] * 100 *2
 matchPercentage = round(matchPercentage)
 name =find_between(file_path[i],"Resumes/",".docx")
 print("----"+name+ " resume matches about " + str(matchPercentage) + "% of the job description.\n")

----Yohan BSA resume matches about 30% of the job description.

----Drakshajavauidev resume matches about 60% of the job description.

----Mounika health care resume resume matches about 25% of the job description.

----Sravani Singirikonda resume matches about 66% of the job description.

----Kumar Raj resume matches about 64% of the job description.

----Jimi_Desai_PM resume matches about 17% of the job description.

----Nikki_Nimmagadd resume matches about 23% of the job description.

----Saiteja G resume matches about 71% of the job description.

----Jaya prakash resume matches about 59% of the job description.

----Raju Goduguchinta_Technical Program Manager resume matches about 27% of the job description.

----KIRAN KUMAR resume matches about 60% of the job description.

----Siddhartha Gandroju resume matches about 61% of the job description.

----Sravani Battu resume matches about 65% of the job description.

----Amar Sr BSA resume matches about 27% of the job description.

----

In [16]:
from sklearn.metrics.pairwise import cosine_similarity

# Print similarity scores
print("\nSimlarity Scores:")
print(cosine_similarity(count_matrix))


Simlarity Scores:
[[1.         0.32500793]
 [0.32500793 1.        ]]
